In [8]:
from riotwatcher import LolWatcher, ApiError
import pandas as pd

# global variables
api_key = 'RGAPI-588159f5-6a25-427d-8254-9a4b64a0557d'
watcher = LolWatcher(api_key)
my_region = 'euw1'


In [9]:
# fetch matchlist by puuid
me = watcher.summoner.by_name(my_region, 'AHAHAHAXD')
# me = watcher.summoner.by_name(my_region, 'daimox')
my_matches = watcher.match.matchlist_by_puuid("EUROPE", me['puuid'])
matches = []
matchTimeLine = []
participants = []


In [10]:
# fetch game, participant and match timeline and transform into dataframe
for matchId in my_matches:
    match_detail = watcher.match.by_id("EUROPE", matchId)
    timeline = watcher.match.timeline_by_match('EUROPE', matchId)

    match = {}
    match['matchId'] = matchId
    match['gameDuration'] = match_detail['info']['gameDuration']
    match['gameType'] = match_detail['info']['gameType']
    match['gameVersion'] = match_detail['info']['gameVersion']
    matches.append(match)

    for index, row in enumerate(match_detail['info']['participants']):
        participants_row = {}
        participants_row['matchId'] = matchId
        participants_row['participant'] = index + 1
        participants_row['summonerName'] = row['summonerName']
        participants_row['teamPosition'] = row['teamPosition']
        participants_row['championName'] = row['championName']
        participants_row['kda'] = f"{row['kills']}/{row['deaths']}/{row['assists']}"
        participants_row['champLevel'] = row['champLevel']
        participants_row['totalDamageToChampions'] = row['totalDamageDealtToChampions']
        participants_row['totalDamageTaken'] = row['totalDamageTaken']
        participants_row['totalHeal'] = row['totalHeal']
        participants_row['totalMinionsKilled'] = row['totalMinionsKilled']
        participants_row['goldEarned'] = row['goldEarned']
        participants_row['summoner1'] = row['summoner1Id']
        participants_row['summoner2'] = row['summoner2Id']
        participants_row['item0'] = row['item0']
        participants_row['item1'] = row['item1']
        participants_row['item2'] = row['item2']
        participants_row['item3'] = row['item3']
        participants_row['item4'] = row['item4']
        participants_row['item5'] = row['item5']
        participants_row['item6'] = row['item6']
        participants_row['win'] = row['win']
        participants.append(participants_row)

    for row in timeline['info']['frames']:
        # frameCounter = 1
        # frame = row["{:02d}".format(frameCounter)]
        # frameCounter += 1
        for frameRow in row['events']:
            timeline_row = {}
            timeline_row['matchId'] = matchId
            eventCounter = 0
            # eventFrame = frameRow["{:02d}".format(eventCounter)]
            eventFrame = frameRow
            eventCounter += 1

            if eventFrame['type'] == 'CHAMPION_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['victimId'] = eventFrame['victimId']
                if 'assistingParticipantIds' in eventFrame:
                    timeline_row['assistingParticipantIds'] = eventFrame['assistingParticipantIds']
                if 'bounty' in eventFrame:
                    timeline_row['bounty'] = eventFrame['bounty']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'BUILDING_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['teamId'] = eventFrame['teamId']
                timeline_row['buildingType'] = eventFrame['buildingType']
                timeline_row['laneType'] = eventFrame['laneType']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'ITEM_PURCHASED':
                timeline_row['type'] = eventFrame['type']
                timeline_row['participantId'] = eventFrame['participantId']
                timeline_row['itemId'] = eventFrame['itemId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'WARD_PLACED':
                timeline_row['type'] = eventFrame['type']
                timeline_row['wardType'] = eventFrame['wardType']
                timeline_row['creatorId'] = eventFrame['creatorId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'WARD_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['wardType'] = eventFrame['wardType']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'LEVEL_UP':
                timeline_row['type'] = eventFrame['type']
                timeline_row['participantId'] = eventFrame['participantId']
                timeline_row['level'] = eventFrame['level']
                timeline_row['timestamp'] = eventFrame['timestamp']
                matchTimeLine.append(timeline_row)
            elif eventFrame['type'] == 'ELITE_MONSTER_KILL':
                timeline_row['type'] = eventFrame['type']
                timeline_row['killerId'] = eventFrame['killerId']
                timeline_row['teamId'] = eventFrame['killerTeamId']
                if 'assistingParticipantIds' in eventFrame:
                    timeline_row['assistingParticipantIds'] = eventFrame['assistingParticipantIds']
                timeline_row['positionX'] = eventFrame['position']['x']
                timeline_row['positionY'] = eventFrame['position']['y']
                timeline_row['timestamp'] = eventFrame['timestamp']
                timeline_row['monsterType'] = eventFrame['monsterType']
                matchTimeLine.append(timeline_row)


participantsDf = pd.DataFrame(participants)
matchesDf = pd.DataFrame(matches)
matchTimelineDf = pd.DataFrame(matchTimeLine)
print(participantsDf)
print(matchesDf)
print(matchTimelineDf)


             matchId  participant      summonerName teamPosition championName  \
0    EUW1_5660215232            1         AHAHAHAXD          TOP       Aatrox   
1    EUW1_5660215232            2      pipigangster       JUNGLE         Udyr   
2    EUW1_5660215232            3  Chinesefastcombo       MIDDLE          Vex   
3    EUW1_5660215232            4       slantedgray       BOTTOM        Vayne   
4    EUW1_5660215232            5            Kelody      UTILITY          Lux   
..               ...          ...               ...          ...          ...   
195  EUW1_5653853007            6     Reincarnation       MIDDLE     Vladimir   
196  EUW1_5653853007            7       Rikoshet437          TOP       Darius   
197  EUW1_5653853007            8       piquefan726       BOTTOM       Lucian   
198  EUW1_5653853007            9           Kamayuq       JUNGLE        Viego   
199  EUW1_5653853007           10             ReIax      UTILITY         Nami   

         kda  champLevel  t

In [11]:
print(timeline['info']['frames'][1])
participantsDf.to_json('participants.json', orient='records')
matchesDf.to_json('matches.json', orient='records')
matchTimelineDf.to_json('matchTimeline.json', orient='records')


{'events': [{'itemId': 1056, 'participantId': 1, 'timestamp': 2737, 'type': 'ITEM_PURCHASED'}, {'itemId': 2003, 'participantId': 1, 'timestamp': 3232, 'type': 'ITEM_PURCHASED'}, {'itemId': 2033, 'participantId': 4, 'timestamp': 3331, 'type': 'ITEM_PURCHASED'}, {'itemId': 2003, 'participantId': 1, 'timestamp': 3364, 'type': 'ITEM_PURCHASED'}, {'itemId': 3340, 'participantId': 1, 'timestamp': 3562, 'type': 'ITEM_PURCHASED'}, {'itemId': 3340, 'participantId': 4, 'timestamp': 3958, 'type': 'ITEM_PURCHASED'}, {'itemId': 1055, 'participantId': 8, 'timestamp': 4719, 'type': 'ITEM_PURCHASED'}, {'itemId': 3340, 'participantId': 8, 'timestamp': 5518, 'type': 'ITEM_PURCHASED'}, {'itemId': 3850, 'participantId': 5, 'timestamp': 5584, 'type': 'ITEM_PURCHASED'}, {'itemId': 2003, 'participantId': 8, 'timestamp': 6046, 'type': 'ITEM_PURCHASED'}, {'itemId': 3340, 'participantId': 5, 'timestamp': 6245, 'type': 'ITEM_PURCHASED'}, {'itemId': 2003, 'participantId': 5, 'timestamp': 6542, 'type': 'ITEM_PURCH